# Advent of Code 2021
## [Day 18: Snailfish](https://adventofcode.com/2021/day/18)

#### Load Data

In [1]:
import json
import math
#from collections import deque
from textwrap import indent
#import weakref
#import treelib

In [2]:
import aocd
input_data = [json.loads(s) for s in aocd.get_data(year=2021, day=18).split('\n')]
input_data[:5]

[[[8, 8], 5],
 [[[[9, 0], 1], 4], [[3, 6], [0, 5]]],
 [[9, [0, [4, 5]]], [1, [[6, 8], 4]]],
 [[8, 7], [[[8, 5], [2, 0]], [[6, 3], [5, 0]]]],
 [[[1, 8], 2], [[[9, 1], [2, 0]], [1, [9, 4]]]]]

### Part 1

In [3]:
class SnailfishNumber(object):
    parent = None
    value = None
    lhs = None
    rhs = None
    
    def __init__(self, value):
        if value is None:
            return
        if type(value) is str:
            value = json.loads(value)
        if type(value) is int:
            self.__class__ = RegularNum
            self.__init__(value)
        elif type(value) is list:
            self.__class__ = PairNum
            self.__init__(value[0], value[1])
        elif issubclass(type(value.__class__), type(SnailfishNumber)):
            # make a deep copy
            self.__class__ = value.__class__
            self.value = value.value
            self.lhs = SnailfishNumber(value.lhs)
            if self.lhs:
                self.lhs.parent = self
            self.rhs = SnailfishNumber(value.rhs)
            if self.rhs:
                self.rhs.parent = self
        else:
            raise TypeError()
            
    def depth(self):
        if self.parent is None:
            return 0
        return 1 + self.parent.depth()

    def __add__(self, rhs: "SnailfishNumber"):
        joined = PairNum(self, rhs)
        joined.reduce()
        return joined
    
# SnailfishNumber([[[[1,2],[3,4]],[[5,6],[7,8]]],9])

In [4]:
class RegularNum(SnailfishNumber):
    def __init__(self, value):
        self.value = value
        
    def __repr__(self):
        return f"{self.value}"
    
    def __add__(self, other):
        if isinstance(other, RegularNum):
            other = other.value
        return RegularNum(self.value + other)

    def __iadd__(self, other):
        if isinstance(other, RegularNum):
            other = other.value
        self.value += other
        return self
    
    def height(self):
        return 0

    def magnitude(self):
        return self.value
    
    def leftmost_leaf(self):
        return self
    
    def rightmost_leaf(self):
        return self
    
r = RegularNum(2)
r += RegularNum(3)
r += 4
r

9

In [5]:
class PairNum(SnailfishNumber):
    def __init__(self, lhs, rhs):
        self.lhs = SnailfishNumber(lhs)
        self.lhs.parent = self
        self.rhs = SnailfishNumber(rhs)
        self.rhs.parent = self
        
    def __repr__(self):
        # repr is one-line and can be parsed
        return f"[{repr(self.lhs)},{repr(self.rhs)}]"
        
    def __str__(self):
        # draw a multi-line tree structure
        lhs = indent(str(self.lhs), '│  ')
        lhs = '├──' + lhs[3:]
        rhs = indent(str(self.rhs), '   ')
        rhs = '└──' + rhs[3:]
        return f"┬{lhs[1:]}\n{rhs}"
    
    def height(self):
        if type(self.lhs) is int and type(self.rhs) is int:
            return 1
        return 1 + max(self.lhs.height(), self.rhs.height())

    def magnitude(self):
        return 3*self.lhs.magnitude() + 2*self.rhs.magnitude()
        
    def leftmost_leaf(self):
        return self.lhs.leftmost_leaf()

    def rightmost_leaf(self):
        return self.rhs.rightmost_leaf()

    def is_simple_pair(self):
        return self.height() == 1

PairNum(8,7)

[8,7]

In [6]:
s = SnailfishNumber([[[[1,2],[3,4]],[[5,6],[7,8]]],9])
print(s)
s

┬──┬──┬──┬──1
│  │  │  └──2
│  │  └──┬──3
│  │     └──4
│  └──┬──┬──5
│     │  └──6
│     └──┬──7
│        └──8
└──9


[[[[1,2],[3,4]],[[5,6],[7,8]]],9]

#### RegularNum.split method

In [7]:
def split(self):
    lhs = RegularNum(math.floor(self.value / 2))
    rhs = RegularNum(math.ceil(self.value / 2))
    lhs.parent = self
    rhs.parent = self

    self.value = None
    self.__class__ = PairNum
    self.__init__(lhs, rhs)
        
RegularNum.split = split
        
r = RegularNum(15)
r.split()
r

[7,8]

In [8]:
def find_split_leaf(self: RegularNum):
    if self.value >= 10:
        return self
    
RegularNum.find_split = find_split_leaf

def find_split_pair(self: PairNum):
    return self.lhs.find_split() or self.rhs.find_split()

PairNum.find_split = find_split_pair

s = SnailfishNumber([[[[0,7],4],[15,[0,13]]],[1,1]])
s.find_split()

15

In [9]:
s.find_split().split()
s

[[[[0,7],4],[[7,8],[0,13]]],[1,1]]

#### PairNum.explode method

In [10]:
def find_leaf_to_right(self):
    cursor = self
    while cursor.parent is not None:
        if cursor.parent.lhs is cursor:
            return cursor.parent.rhs.leftmost_leaf()
        cursor = cursor.parent

SnailfishNumber.find_leaf_to_right = find_leaf_to_right

def find_leaf_to_left(self):
    cursor = self
    while cursor.parent is not None:
        if cursor.parent.rhs is cursor:
            return cursor.parent.lhs.rightmost_leaf()
        cursor = cursor.parent

SnailfishNumber.find_leaf_to_left = find_leaf_to_left

s = SnailfishNumber([[[[1,2],[3,4]],[[5,6],[7,8]]],9])
s.lhs.lhs.rhs, s.lhs.lhs.rhs.find_leaf_to_right()

([3,4], 5)

In [11]:
def explode(self):
    lhs, rhs = self.lhs, self.rhs
    
    leftie = self.find_leaf_to_left()
    if leftie:
        leftie += self.lhs

    rightie = self.find_leaf_to_right()
    if rightie:
        rightie += self.rhs

    self.lhs = None
    self.rhs = None
    self.__class__ = RegularNum
    self.__init__(0)

PairNum.explode = explode
s = SnailfishNumber([[1,2],[3,4]])
s.lhs.explode()
s

[0,[5,4]]

In [12]:
def find_explode(self):
    if not isinstance(self, PairNum):
        return None
    if self.is_simple_pair() and self.depth() >= 4:
        return self
    found = self.lhs.find_explode() or self.rhs.find_explode()
    return found

SnailfishNumber.find_explode = find_explode

s = SnailfishNumber([5,[[[[[9,8],1],2],3],4]])
s.find_explode()

[9,8]

In [13]:
s.find_explode().explode()
s

[14,[[[[0,9],2],3],4]]

In [14]:
s = SnailfishNumber([[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]])
s.find_explode()

[4,3]

#### Reduce Operation

In [15]:
def reduce(self, verbose=False):
    """reduce it"""
    if verbose:
        print(f"after addition: {repr(self)}")
    done = False
    while not done:
        done = True
        to_explode = self.find_explode()
        if to_explode:
            done = False
            to_explode.explode()
            if verbose:
                print(f"after explode:  {repr(self)}")
            continue
        to_split = self.find_split()
        if to_split:
            done = False
            to_split.split()
            if verbose:
                print(f"after split:    {repr(self)}")

SnailfishNumber.reduce = reduce

s = SnailfishNumber([[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]])
s.reduce(verbose=True)

after addition: [[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
after explode:  [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
after explode:  [[[[0,7],4],[15,[0,13]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
after explode:  [[[[0,7],4],[[7,8],[6,0]]],[8,1]]


In [16]:
s = SnailfishNumber([[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]])
print(s)
s.reduce()
print(s)

┬──┬──┬──┬──┬──4
│  │  │  │  └──3
│  │  │  └──4
│  │  └──4
│  └──┬──7
│     └──┬──┬──8
│        │  └──4
│        └──9
└──┬──1
   └──1
┬──┬──┬──┬──0
│  │  │  └──7
│  │  └──4
│  └──┬──┬──7
│     │  └──8
│     └──┬──6
│        └──0
└──┬──8
   └──1


#### SnailfishNum Add operator

In [17]:
s1 = SnailfishNumber([[[[4,3],4],4],[7,[[8,4],9]]])
s2 = SnailfishNumber([1,1])
s1 + s2

[[[[0,7],4],[[7,8],[6,0]]],[8,1]]

#### Sum of lists

In [18]:
test_data = [
    [1,1],
    [2,2],
    [3,3],
    [4,4],
    [5,5],
    [6,6]
]

In [19]:
def homework(input_data, verbose=False):
    result = SnailfishNumber(input_data[0])
    for n in input_data[1:]:
        num = SnailfishNumber(n)
        if verbose:
            print(" ", repr(result))
            print("+", repr(num))
        result = result + num
        if verbose:
            print("=", repr(result))
            print()
    return result
homework(test_data, verbose=False)

[[[[5,0],[7,4]],[5,5]],[6,6]]

In [20]:
test_data = [
    [[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]],
    [7,[[[3,7],[4,3]],[[6,3],[8,8]]]],
    [[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]],
    [[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]],
    [7,[5,[[3,8],[1,4]]]],
    [[2,[2,2]],[8,[8,1]]],
    [2,9],
    [1,[[[9,3],9],[[9,0],[0,7]]]],
    [[[5,[7,4]],7],1],
    [[[[4,2],2],6],[8,7]]
]
homework(test_data)

[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]

In [21]:
SnailfishNumber(test_data[0]) + SnailfishNumber(test_data[1])

[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]

#### Part 1 Answer
Add up all of the snailfish numbers from the homework assignment in the order they appear.  
**What is the magnitude of the final sum?**

In [22]:
homework(input_data).magnitude()

3725

### Part 2

In [23]:
test_data = [
    [[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]],
    [[[5,[2,8]],4],[5,[[9,9],0]]],
    [6,[[[6,2],[5,6]],[[7,6],[4,7]]]],
    [[[6,[0,7]],[0,9]],[4,[9,[9,0]]]],
    [[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]],
    [[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]],
    [[[[5,4],[7,7]],8],[[8,3],8]],
    [[9,3],[[9,9],[6,[4,9]]]],
    [[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]],
    [[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]    
]

In [24]:
def largest_mag(input_data):
    largest_seen = 0
    for i in range(len(input_data)):
        for j in range(len(input_data)):
            if i == j:
                continue
            num_i = SnailfishNumber(input_data[i])
            num_j = SnailfishNumber(input_data[j])
            mag = (num_i + num_j).magnitude()
            if mag > largest_seen:
                largest_seen = mag
    return largest_seen
largest_mag(test_data)

3993

#### Part 2 Answer
**What is the largest magnitude of any sum of two different snailfish numbers from the homework assignment?**

In [25]:
largest_mag(input_data)

4832